# Training Deep Neural Networks

In [23]:
import tensorflow as tf
from tensorflow import keras

In [24]:
keras.layers.Dense(10, activation="relu", kernel_initializer=keras.initializers.he_normal)

In [25]:
he_avg_init = keras.initializers.VarianceScaling(scale=2, mode="fan_avg",
                                                 distribution='uniform')
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

In [26]:
keras.layers.Dense(10, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal)

In [27]:
model = keras.models.Sequential([
 keras.layers.Flatten(input_shape=[28, 28]),
 keras.layers.BatchNormalization(),
 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
 keras.layers.BatchNormalization(),
 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
 keras.layers.BatchNormalization(),
 keras.layers.Dense(10, activation="softmax")
])

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_6 (Bat  (None, 784)               3136      
 chNormalization)                                                
                                                                 
 dense_18 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_7 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_19 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_8 (Bat  (None, 100)              

In [29]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_6/gamma:0', True),
 ('batch_normalization_6/beta:0', True),
 ('batch_normalization_6/moving_mean:0', False),
 ('batch_normalization_6/moving_variance:0', False)]

In [30]:
model.layers[1].updates

/tmp/ipykernel_669/3873162892.py:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [31]:
model = keras.models.Sequential([
 keras.layers.Flatten(input_shape=[28, 28]),
 keras.layers.BatchNormalization(),
 keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
 keras.layers.BatchNormalization(),
 keras.layers.Activation("elu"),
 keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
 keras.layers.BatchNormalization(),
 keras.layers.Activation("elu"),
 keras.layers.Dense(10, activation="softmax")
])

In [32]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss=keras.losses.mse, optimizer=optimizer)

In [33]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [34]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [35]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [36]:
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-4)

In [37]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

In [38]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

In [39]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# history = model.fit(X_train_scaled, ..., callbacks=[lr_scheduler])

In [40]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [41]:
X_train = []
s = 20 * len(X_train) // 32
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

In [42]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer=keras.initializers.he_normal,
                           kernel_regularizer=keras.regularizers.l2(0.1))

In [43]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation=keras.activations.softmax,
                     kernel_initializer=keras.initializers.glorot_uniform)
])

In [44]:
model = keras.models.Sequential([
 keras.layers.Flatten(input_shape=[28, 28]),
 keras.layers.Dropout(rate=0.2),
 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
 keras.layers.Dropout(rate=0.2),
 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
 keras.layers.Dropout(rate=0.2),
 keras.layers.Dense(10, activation="softmax")
])

# Exercises

In [46]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [51]:
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=X_train_full.shape[1:]))

for _ in range(20):
    model.add(keras.layers.Dense(100,
                                activation=keras.activations.elu,
                                kernel_initializer=keras.initializers.he_normal))

In [52]:
model.add(keras.layers.Dense(10, activation=keras.activations.softmax))

In [53]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
             optimizer=optimizer,
             metrics=["accuracy"])

In [54]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [55]:
import os

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [60]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2698), started 0:01:34 ago. (Use '!kill 2698' to kill it.)

In [61]:
model.fit(X_train, y_train, epochs=100,
          validation_data=[X_valid, y_valid],
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 11s 5ms/step - loss: 3.8410 - accuracy: 0.1852 - val_loss: 2.3325 - val_accuracy: 0.1846
Epoch 2/100
  22/1407 [..............................] - ETA: 6s - loss: 2.1902 - accuracy: 0.2259

/home/mihnea/personal/ai/env/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 7s 5ms/step - loss: 2.0609 - accuracy: 0.2551 - val_loss: 2.0410 - val_accuracy: 0.2778
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.9518 - accuracy: 0.2946 - val_loss: 1.9482 - val_accuracy: 0.3022
Epoch 4/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.8789 - accuracy: 0.3167 - val_loss: 1.8658 - val_accuracy: 0.3376
Epoch 5/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.8216 - accuracy: 0.3433 - val_loss: 1.8172 - val_accuracy: 0.3414
Epoch 6/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.7683 - accuracy: 0.3622 - val_loss: 1.7657 - val_accuracy: 0.3516
Epoch 7/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.7224 - accuracy: 0.3773 - val_loss: 1.7157 - val_accuracy: 0.3798
Epoch 8/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6868 - accuracy: 0.3928 - val_loss: 1.7044 - val_accuracy: 0.38

In [62]:
best_model = keras.models.load_model("my_cifar10_model.h5")

In [63]:
best_model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5246 - accuracy: 0.4738


[1.5246142148971558, 0.47380000352859497]

In [65]:
import numpy as np

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 23s 10ms/step - loss: 1.8399 - accuracy: 0.3390 - val_loss: 1.6501 - val_accuracy: 0.4188
Epoch 2/100
   7/1407 [..............................] - ETA: 12s - loss: 1.6666 - accuracy: 0.4062

/home/mihnea/personal/ai/env/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 13s 10ms/step - loss: 1.6686 - accuracy: 0.4061 - val_loss: 1.5798 - val_accuracy: 0.4302
Epoch 3/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.6015 - accuracy: 0.4323 - val_loss: 1.5385 - val_accuracy: 0.4532
Epoch 4/100
1407/1407 [==============================] - 13s 10ms/step - loss: 1.5501 - accuracy: 0.4483 - val_loss: 1.5115 - val_accuracy: 0.4632
Epoch 5/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5073 - accuracy: 0.4652 - val_loss: 1.4626 - val_accuracy: 0.4788
Epoch 6/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4687 - accuracy: 0.4812 - val_loss: 1.4205 - val_accuracy: 0.4988
Epoch 7/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4348 - accuracy: 0.4901 - val_loss: 1.4416 - val_accuracy: 0.4912
Epoch 8/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4057 - accuracy: 0.5029 - val_loss: 1.4035 - val_acc

[1.3206573724746704, 0.545199990272522]